In [2]:
import pandas as pd
import numpy as np

In [3]:
df1 = pd.read_csv("data.csv")

In [4]:
df1.head()

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"


In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

In [ ]:
# Рассмотрим какие данные находятся в колонках 

#colname = 'propertyType2'
#colname ='street'
#colname ='baths'
#colname ='fireplace' # ???
#colname ='homeFacts'
#colname ='schools'
#colname ='sqft'
#colname ='beds'
#colname ='PrivatePool'
#colname ='private pool'
#colname ='stories' # ????
#colname ='MlsId'
#colname ='target2'
#colname ='status2'
#colname = 'propertyType2'
colname = 'Pricesqft'
baths = pd.DataFrame(df1[colname].value_counts())
#print(type(baths))
baths =baths.sort_index(ascending=False)
for i in baths.index:
    print(i)

Данные очень грязные необходимо сделать предварительную обработку.
Создадим функции по обработке данных в колонках.

In [ ]:
def replacebath(x):
    x = str(x).upper()
    х = str(x).strip()
    x = x.replace(str('Bathrooms: ').upper() , '')
    x = x.replace('+' , '')
    x = x.replace("'" , '')    
    x = str(x).strip().split(' ')[0].split('-')[0].replace(',','.')
    if str(x).isnumeric():
        return int(x)
    else:
        return 0

In [ ]:
df1['baths2'] = df1.apply(lambda x: replacebath(x['baths']), axis = 1)

In [ ]:
def replacesqft(x):
    x = str(x).upper()
    х = str(x).strip()
    x = x.replace(str('Total interior livable area:').upper() , '')
    x = x.replace(',' , '')
    x = x.replace('+' , '')
    x = x.replace("'" , '')
    x = x.replace(str('sqft').upper() , '')
       
    x = str(x).strip()
    return x

In [ ]:
df1['sqft2'] = df1.apply(lambda x: replacesqft(x['sqft']), axis = 1)

In [ ]:
def replacetarget(x):
    x = str(x).upper()
    х = str(x).strip()
    x = x.replace('$' , '')
    x = x.replace(',' , '')
    x = x.replace('+' , '')
    x = x.replace('/MO' , '')
    x = x.replace("'" , '')   
    x = str(x).strip()
    return x

In [ ]:
df1['target2'] = df1.apply(lambda x: replacetarget(x['target']), axis = 1)

In [ ]:
def replacestatus(x):
    if x == np.nan:
        return x
    
    x = str(x).upper()
    х = str(x).strip()
    if x == 'C':
        x ='CONTINUE SHOW'
        
    x = x.replace('UNDER CONTRACT SHOW','UNDER CONTRACT - SHOW')
    x = x.replace('UNDER CONTRACT   SHOWING','UNDER CONTRACT - SHOW')
    x = x.replace('RE ACTIVATED','REACTIVATED')
    x = x.replace('CONTINGENT   SHOW'     ,'CONTINGENT SHOW')
    x = x.replace('/ AUCTION' , 'AUCTION')
    x = x.replace('A ACTIVE' , 'ACTIVE')
    x = x.replace('ACTIVE/CONTINGENT', 'ACTIVE CONTINGENT')
    x = x.replace('C CONTINUE SHOW', 'CONTINUE SHOW')
    x = x.replace('U UNDER CONTRACT', 'UNDER CONTRACT')
    x = x.replace('P PENDING SALE', 'PENDING')
    x = x.replace('NEW CONSTRUCTION','NEW')
    x = x.replace('PENDING   CONTINUE TO SHOW', 'PENDING CONTINUE TO SHOW')
    
    x = x.replace("'" , '')
    х = str(x).strip()
    
    if x.find('COMING SOON') != -1 :
           x = 'COMING SOON'

    if x.find('SOLD') != -1 :
           x = 'SOLD'
             
    x = str(x).strip()
    return x

In [ ]:
df1['status2'] = df1.apply(lambda x: replacestatus(x['status']), axis = 1)

In [ ]:
def replaceProperty(x):
    x = str(x).upper()
    х = str(x).strip()
    x = x.replace('SINGLE-FAMILY HOME','SINGLE FAMILY')
    x = x.replace('SINGLE FAMILY HOME','SINGLE FAMILY')
    x = x.replace('MULTI-FAMILY HOME','MULTI FAMILY')
    x = x.replace('MULTI-FAMILY'     ,'MULTI FAMILY')
    x = x.replace('SPLIT-LEVEL' , 'SPLIT LEVEL')
    x = x.replace('OTHER - SEE REMARKS' , 'OTHER (SEE REMARKS)')
    x = x.replace('OTHER STYLE' , 'OTHER')
    x = x.replace('PATIO HOME' , 'PATIO')
    x = x.replace('MOBILE / MANUFACTURED' , 'MOBILE/MANUFACTURED')
    x = x.replace('MID-CENTURY MODERN'  , 'MID CENTURY MODERN')
    x = x.replace('LOG HOME' , 'LOG')
    x = x.replace('HISTORICAL' , 'HISTORIC')
    x = x.replace('HIGH RISE' , 'HIGH-RISE')
    x = x.replace('FARM HOUSE' , 'FARMHOUSE')
    x = x.replace('BILEVEL' , 'BI-LEVEL')
    x = x.replace('CARRIAGE HOUSE' , 'CARRIAGE')
    x = x.replace('CUSTOM HOME' , 'CUSTOM')
    x = x.replace("'" , '') 
    х = str(x).strip()
    return x #.split(',')[0]

In [ ]:
df1['propertyType2'] = df1.apply(lambda x: replaceProperty(x['propertyType']), axis = 1)

В колонке homeFacts храняться данные о  год постройки, паркинге, кухне, стоимости за квадратный фут (sqft), размере дома
{'atAGlanceFacts':
[{'factValue': '1961', 'factLabel': 'Year built'}, 
 {'factValue': '1967', 'factLabel': 'Remodeled year'}, 
 {'factValue': 'Forced Air', 'factLabel': 'Heating'}, 
 {'factValue': 'Central', 'factLabel': 'Cooling'}, 
 {'factValue': 'Attached Garage', 'factLabel': 'Parking'}, 
 {'factValue': '8,626 sqft', 'factLabel': 'lotsize'}, 
 {'factValue': '$965/sqft', 'factLabel': 'Price/sqft'}]}

вынесем эти данные в новые колонки

In [ ]:
def add_homeFacts(x):
    ls1 =x.replace("{'atAGlanceFacts': [", "").replace(']}','').replace("{'factValue':",'').replace("'factLabel':",'').split('}')
    _l =[np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]
    if len(ls1)>=7 :
            _l[0]=ls1[0].split(',')[0] #Yearbuilt
            _l[1]=ls1[1].split(',')[1]  # Remodeledyear
            _l[2] = ls1[2].split(',')[1] # Heating
            _l[3] = ls1[3].split(',')[1] # Cooling
            _l[4] = ls1[4].split(',')[1] # Parking
            _l[5] = ls1[5].split(',')[1].replace("sqft",'') # lotsize
            _l[6] = ls1[6].split(',')[1].replace("/sqft",'').replace("$",'').replace( '/ Sq. Ft.','') # Pricesqft
    return _l

In [ ]:
df1['Yearbuilt'] = df1.apply(lambda x: add_homeFacts(x['homeFacts'])[0], axis = 1)
df1['Remodeledyear'] = df1.apply(lambda x: add_homeFacts(x['homeFacts'])[1], axis = 1)
df1['Heating'] = df1.apply(lambda x: add_homeFacts(x['homeFacts'])[2], axis = 1)
df1['Cooling'] = df1.apply(lambda x: add_homeFacts(x['homeFacts'])[3], axis = 1)
df1['Parking'] = df1.apply(lambda x: add_homeFacts(x['homeFacts'])[4], axis = 1)
df1['lotsize'] = df1.apply(lambda x: add_homeFacts(x['homeFacts'])[5], axis = 1)
df1['Pricesqft'] = df1.apply(lambda x: add_homeFacts(x['homeFacts'])[6], axis = 1)

In [ ]:
def replacePricesqft(x):
    x = str(x).upper()
    х = str(x).strip()
    x = x.replace(str('/ Sq. Ft.').upper() , '')
    x = x.replace(str('Sq. Ft.').upper() , '')
    x = x.replace(str('lot').upper() , '')
    x = x.replace(str('acres').upper() , '')
    x = x.replace(str('-').upper() , '')
    x = x.replace(str('Acres').upper() , '')
    x = x.replace('+' , '')
    x = x.replace("'" , '') 
    x = x.replace(" " , '')
    
    x = str(x).strip()
    if str(x).isnumeric():
        return float(x)
    else:
        return 0

In [ ]:
df1['Pricesqft2'] = df1.apply(lambda x: replacePricesqft(x['Pricesqft']), axis = 1)
df1['lotsize'] = df1.apply(lambda x: replacePricesqft(x['lotsize']), axis = 1)

In [ ]:
В колонке schools есть список школ с рейтингом, расстоянием до них и оценкой школы
"[{'rating': ['8/10', '4/10', '8/10'], 'data': {'Distance': ['1.19mi', '2.06mi', '2.63mi'], 'Grades': ['6-8', 'K-5', '9-12']}, 'name': ['Paul Revere Middle School', 'Brentwood Science School', 'Palisades Charter High School']}]"

Для анализа школы возьмем данные по ближайшей. В качестве улучшения модели можно добавить среднее, общее количество.

In [ ]:
def add_schools(x):
    _lsp= x.replace(' ','').replace("'rating':",'|').replace("'data':",'|').replace("'name':",'|').replace('[{','').replace('}]','').split('|')
    #print (_lsp)
    _rating=_lsp[1].replace(']','').replace('[','').split(',')
    _llsp2= _lsp[2].replace("'Distance':",'|').replace("'Grades':",'|').split('|')
    _Distance = _llsp2[1].replace(']','').replace('[','').replace('mi','').replace("'",'').split(',')
    _Grades = _llsp2[2].replace(']}','').replace('[','').split(',')

    _minDistance = 0.
    _minrating = np.nan
    _minGrades = 'NR'

    for i in range(0,len(_Distance)):
        #print (_Distance[i])
        if _Distance[i] == '':
            continue
        
        if _minDistance == 0:
            _minDistance = float(_Distance[i])
            _minrating = _rating[i]
            if i <len(_Grades):
                _minGrades = _Grades[i]
            else:
                _minGrades = 'NR'
        
        if float(_Distance[i]) < _minDistance:
            _minDistance =float(_Distance[i])
            _minrating = _rating[i]
            if i <len(_Grades):
                _minGrades = _Grades[i]
            else:
                _minGrades = 'NR'

    return (_minrating,_minDistance,  _minGrades)   

In [ ]:
df1['rating1'] = df1.apply(lambda x: add_schools(x['schools'])[0], axis = 1)
df1['Distance1'] = df1.apply(lambda x: add_schools(x['schools'])[1], axis = 1)
df1['Grades1'] = df1.apply(lambda x: add_schools(x['schools'])[2], axis = 1)

In [ ]:
df1.target2.dropna(inplace=True)

Сохраним в файл.

In [ ]:
df1.to_csv('data_all_small_3.csv')